In [ ]:
###Python Packages###

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib.colors import LogNorm
import math as mth

import timeit
import Double_Null_utils as dnu
from scipy import optimize
import tables
from mpmath import *


In [ ]:
###Initial Values###

M0=1.0
Q=0.95
Lambda=0.0000
scalarfield=False
vscalarfield=False


In [ ]:
Tol=1
N=50
Nul=100
Nup=10**(40)
scal=1
umax=7.5
vmax=100
datatype=np.float64


#ru0=5.0
#dr0v=.4
ru0=2.0
#dr0v=0.045
dr0v=1/(4*.375)*(1-2*M0/ru0+Q**2/ru0**2)
#dr0v=0.045

bdytype="stan"
scaltostan=False
solveaffine=False

Elist=[1]
#Elist=[1,2,4,8,16]

#uloc=[1/8,1/4,3/8,1/2,5/8]
#uloc=[1/5,2/5,3/5,4/5,9/10]
uloc=[8.2/10]

In [ ]:
###Defining Needed Constants###

scalf=float(scal)
u0=0.0
v0=0.0
rv0=ru0
phiu0=0.0
phiv0=0.0

if M0==0.0:
    dv0=1/N    
else:
    dv0=M0/N
    


def fr(r,M0,Q,Lambda):
    return 1-2*M0/r+Q**2.0/r**2.0-Lambda*r**2.0/3.0

if Lambda>0 or Lambda<0:
    sol = optimize.root(fr,[0.687,1.3,50.0],args=(M0,Q,Lambda), method='hybr')
    rminus=sol.x[0]
    rplus=sol.x[1]
    rcosm=sol.x[2]
else:
    rplus=M0+(M0**2-Q**2)**(.5)
    rminus=M0-(M0**2-Q**2)**(.5)
    rcosm=0.0
    

wHD=True 

print("r+: "+str(rplus))
print("r-: "+str(rminus))
print("rc: "+str(rcosm))

###Predicting Run time###

###timer
start = timeit.default_timer()

Nu=Nul*umax*100#int(umax/du0)
Nv=int(vmax/dv0)
#print("Number of points for lowest iteration is "+str(Nu*Nv)+","+str(Nu)+"X"+str(Nv))
print("Number of points for highest iteration is "+str(Nu*Nv*max(Elist)**2)+","+str(Nu*max(Elist))+"X"+str(Nv*max(Elist)))

numpoints=0.0
for i in range(0,len(Elist)):
    numpoints=Elist[i]**2.0*Nu*Nv+numpoints
print("Total number of points is "+str(int(numpoints)))

predtime=numpoints/(78000)

print("Runtime: about "+str(format(predtime/60,'.2f'))+" minutes")


In [ ]:
##################################
###Applying Boundary Conditions###

Emax=max([Elist])


rnpf=np.zeros((2,Nv),dtype=datatype)
signpf=np.zeros((2,Nv),dtype=datatype)
phinpf=np.zeros((2,Nv),dtype=datatype)
massnpf=np.full((2,Nv),M0)#np.zeros((2,Nv))
drunpv=np.full((1,Nv-1),-1/(4*dr0v)*(1-2*M0/ru0+Q**2/ru0**2))
 


rnpf[0], signpf[0], phinpf[0] = dnu.boundaryv(scal,bdytype,Nv,ru0,dr0v,dv0,vmax,M0,Q,Lambda,scalarfield,datatype)




In [ ]:
drunptemp=np.empty((Nv))*np.nan
#dsigunp=np.empty((Nu))*np.nan


A=.115
u1=9.5
u2=10

###Applying Propagation Algorithm###

urange=np.array([0.0],dtype=datatype)
drunp=np.array([0.0],dtype=datatype)
dsigunp=np.array([0.0],dtype=datatype)
dphinpuf=np.array([0.0],dtype=datatype)
massnpf[0][0]=1.0

drunp[0]=-mth.exp(signpf[0][0])/(4.0*dr0v)*(1-2*M0/ru0+Q**2.0/ru0**2.0-Lambda*ru0**2/3)
dsigunpvalue=100*np.exp(np.nanmax(signpf))
drunptemp=drunp[0]
    

i=0
dv=dv0
if M0>0:
    du0=M0/N
elif M0==0:
    du0=1/N

du=du0

if bdytype=="edd" or bdytype=="fulledd":
    bdyvalue=1.0
    dsigunp[0]=2*(3*Q**2-3*M0*ru0+ru0**4*Lambda)/(ru0*(-3*Q**2+ru0*(6*M0-3*ru0+ru0**3*Lambda)))*drunp[0]
    #dsigunp[0]=2.0*(M0*rnpf[0]-Q**2.0)/(rnpf[0]*(Q**2.0+rnpf[0]*(-2*M0+rnpf[0])))*drunp[0]
else:
    bdyvalue=0.0
    dsigunp[0]=0.0
 
atom = tables.Float64Atom()

rnpfile = tables.open_file('rnp.h5', mode='w',encoding="utf8") 
rnp = rnpfile.create_earray(rnpfile.root, 'data', atom, (0, Nv))

signpfile = tables.open_file('signp.h5', mode='w',encoding="utf8") 
signp = signpfile.create_earray(signpfile.root, 'data', atom, (0, Nv))
phinpfile = tables.open_file('phinp.h5', mode='w',encoding="utf8") 
phinp = phinpfile.create_earray(phinpfile.root, 'data', atom, (0, Nv))
massnpfile = tables.open_file('massnp.h5', mode='w',encoding="utf8") 
massnp = massnpfile.create_earray(massnpfile.root, 'data', atom, (0, Nv))
    
 
#rtemp=rnpf[0]   
#print(rtemp)
rnp.append(rnpf[0].reshape(1,Nv))
signp.append(signpf[0].reshape(1,Nv))
phinp.append(phinpf[0].reshape(1,Nv))
massnp.append(massnpf[0].reshape(1,Nv))

drunp=np.append(drunp,np.nan)
dsigunp=np.append(dsigunp,np.nan)
dphinpuf=np.append(dphinpuf,np.nan)

TempTol=1 
TempTolv=1
Eres=False
while max(urange)<umax and i<50000:
    
    #rnpf=np.insert(rnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #phinpf=np.insert(phinpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #signpf=np.insert(signpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #massnpf=np.insert(massnpf,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    
    #drunp=np.append(drunp,np.nan)
    #dsigunp=np.append(dsigunp,np.nan)
    #dphinpuf=np.append(dphinpuf,np.nan)
    
    du=du0*Tol*TempTol/max([abs(dsigunpvalue),abs(drunptemp)])
    #du=abs(du0*Tol*TempTol/mth.exp(np.nanmax(signpf[0,:])))
    #dui=abs(mth.exp(np.nanmax(signpf[0,:]))/(du0*Tol))
    
    if du<1/Nup*1.1:
        du=1/Nup
     
    
    if du>=1/Nul:
        #print('r='+str(rnpf[i][0]))
        if rnpf[1][0]<rplus:
            du=1/(1*Nul)
            #print(rplus)
        else:
            du=1/Nul
    
    
    
    
    print("%.24f" % urange[i])
    print("%.24f" % du)
    
    if urange[i]>u1 and urange[i]<u2 and vscalarfield==True:
        u=urange[i]+du
        phinpf[1][0]=A*64*(u-u1)**3.0*(u2-u)**3.0/(u2-u1)**6.0
        dphinpuf[i]=192*A*(u-u1)**2.0*(u-u2)**2.0*(-2*u+u1+u2)/(u1-u2)**6.0 
    else:
        phinpf[1][0]=0.0
        dphinpuf[i+1]=0.0
    
    #print(drunp,dsigunp,dphinpuf)
    
    
    rnpf[1][0]=rnpf[0][0]+du*drunp[i]
    drunp[i+1]=drunp[i]+du*(drunp[i]*dsigunp[i]-rnpf[0][0]*dphinpuf[i]**2.0)
    
    
    
    signpf[1][0]=(signpf[0][0]+du*dsigunp[i])*(1.0-bdyvalue)+np.log(abs(1.0-2.0*M0/rnpf[1][0]+Q**2.0/rnpf[1][0]**2.0))*bdyvalue
    #dsigunp[i+1]=2.0*(M0*rnpf[1][0]-Q**2.0)/(rnpf[1][0]*(Q**2.0+rnpf[1][0]*(-2*M0+rnpf[1][0])))*drunp[i+1]*bdyvalue
    dsigunp[i+1]=2*(3*Q**2-3*M0*rnpf[1][0]+rnpf[1][0]**4*Lambda)/(rnpf[1][0]*(-3*Q**2+rnpf[1][0]*(6*M0-3*rnpf[1][0]+rnpf[1][0]**3*Lambda)))*drunp[i+1]*bdyvalue

    massnpf[1][0]=M0
    
    
    

    
    for j in range(0,Nv-1):
        answer=dnu.x4giveralt(0,j,du,dv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
        rnpf[1][j+1]=answer[0]
        phinpf[1][j+1]=answer[1]
        signpf[1][j+1]=answer[2]
        massnpf[1][j+1]=answer[3]
        #drunptemp[j+1]=answer[4]
        drunptemp=answer[4]
        dsigunpvalue=answer[5]
        exterm1=answer[6]
        exterm2=answer[7]
        exterm3=answer[8]
    
            
            
    #if any(abs((np.exp(signpf[0][:])-np.exp(signpf[1][:]))/np.exp(signpf[0][:]))>0.01):  
    if float(dsigunpvalue*du)>0.1:
        TempTol=TempTol/2
        Eres=True
    else:
        rnp.append(rnpf[1].reshape(1,Nv))
        signp.append(signpf[1].reshape(1,Nv))
        phinp.append(phinpf[1].reshape(1,Nv))
        massnp.append(massnpf[1].reshape(1,Nv))
    
        drunp=np.append(drunp,np.nan)
        dsigunp=np.append(dsigunp,np.nan)
        dphinpuf=np.append(dphinpuf,np.nan)
    
        rnpf[0,:]=rnpf[1,:]
        phinpf[0,:]=phinpf[1,:]
        signpf[0,:]=signpf[1,:]
        massnpf[0,:]=massnpf[1,:]
    
        urange=np.append(urange,urange[i]+du)
        
        if Eres==True:
            TempTol=2*TempTol
        else:
            TempTol=1
        Eres=False
        i+=1
    
    print(TempTol)
    print(exterm1)
    print(exterm2)
    print(exterm3)
    print('---')
    
    #drunp=np.insert(drunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dsigunp=np.insert(dsigunp,(1)*Nv,np.empty((Nv))*np.nan).reshape(i+2,Nv)
    #dphinpuf=np.insert(dphinpuf,(1),np.empty((1))*np.nan).reshape(i+2)
    #print(urange)
     
    #print(urange)
  
    
    #du=abs(du0*Tol/np.nanmin(drunptemp))

    
    
    
    
    if rnpf[1][0]<0.0 or np.isnan(du):
        break
   
    #dumaxlist=[]
    
    
    #if i>1000:
        #break


In [ ]:
rnpfile.close()
signpfile.close()
phinpfile.close()
massnpfile.close()

In [ ]:
Nu=i+2

print(Nu)
#rnpf[i+1][j+1]=answer[0]
#phinpf[i+1][j+1]=answer[1]
#signpf[i+1][j+1]=answer[2]
#massnpf[i+1][j+1]=answer[3]
#drunp[i+1][j+1]=answer[4]
#dsigunp[i+1][j+1]=answer[5]

In [ ]:
#print(rnpf[1,0])

In [ ]:
#print(dnu.x4giveralt(0,0,du,dv,rnpf,phinpf,signpf,Q,Lambda))

In [ ]:
#print(rnpf)

In [ ]:
vrange=np.arange(0,vmax,dv0)
#print(urange)

In [ ]:
switch=0.0
np.save('rnputil',np.array([Nu*Nv,Nu,Nv,ru0,dr0v,M0,Q,Lambda,wHD,switch]))
np.save('urange',urange)
np.save('vrange',vrange)

In [ ]:
stop = timeit.default_timer()

In [ ]:
acttime=stop - start

numpoints=Nu*Nv



print(str(acttime)+' seconds')
print(str((acttime)/60)+' minutes')
print(str((acttime)/3600)+' hours')

print(str(numpoints)+' points')

print(str((numpoints)/(stop - start))+' points per second') 

exttimef=(acttime-predtime)/60

exttime=format(abs((acttime-predtime)/60),'.2f')
if exttimef>0.0:
    print("Took "+str(exttime)+" more minutes")
if exttimef<0.0:
    print("Took "+str(exttime)+" less minutes")
if exttimef==0.0:
    print("Took exactly the right time!!!!")

conv=False
j=0
tempanswer=[]
    while conv==False:
        dv=TempTolv*dv0
        jcount=1
        while j<Nv-1:
            j1=mth.ceil(jcount*TempTolv)
            j2=mth.floor(jcount*TempTolv)
            answer=dnu.x4giveralt(0,j+mth.ceil((jcount-1)*TempTolv),du,dv*TempTolv,rnpf,phinpf,signpf,massnpf,M0,Q,Lambda,datatype)
            rnpf[1][j+j1]=answer[0]
            phinpf[1][j+j1]=answer[1]
            signpf[1][j+j1]=answer[2]
            massnpf[1][j+j1]=answer[3]
            #drunptemp[j+1]=answer[4]
            dsigunpv=answer[5]
            jcount=(jcount+1)**(0**j2)
            j=j+j2
        
        if np.abs(tempanswer[0]-answer[0])/answer[0]>0.01 or not tempanswer:
            tempanswer=answer
            TempTol=TempTolv/2
            
        else:
            conv=True
            